# Lab 6

Advanced APIs for Scale - Batching

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv(override=True)

openai = OpenAI()


In [2]:
questions = [
    "What is the capital of France?",
    "What is the capital of Germany?",
    "What is the capital of Italy?",
    "What is the capital of Spain?",
    "What is the capital of Portugal?",
    "What is the capital of Greece?",
    "What is the capital of Turkey?",
    "What is the capital of Egypt?",
    "What is the capital of South Africa?",
]

## First step - Generate a jsonl file - json line - good for streaming and batch

With this format:

```jsonl
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
```

In [3]:
#generate one line to fill the questions from the list
def make_line(index):
    question = questions[index]
    return f'{{"custom_id": "request-{index}", "method": "POST", "url": "/v1/chat/completions", "body": {{"model": "gpt-4.1-nano", "messages": [{{"role": "user", "content": "{question}"}}],"max_tokens": 1000}}}}'


make_line(0)

'{"custom_id": "request-0", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4.1-nano", "messages": [{"role": "user", "content": "What is the capital of France?"}],"max_tokens": 1000}}'

In [5]:
#create new jsonl file in the folder

with open("questions.jsonl", "w", encoding="utf-8") as f:
    for i in range(len(questions)):
        f.write(make_line(i) + "\n")

## Second step - upload the file

In [6]:
#create a file with file object and id uploaded
questions_file = openai.files.create(file=open("questions.jsonl", "rb"), purpose="batch")
questions_file

FileObject(id='file-5zSDHwyFTsKjjFQTfhxG2t', bytes=1878, created_at=1750718604, filename='questions.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)

## Third step - make the batch

In [7]:
#completion_window='24h' means openai you hvae 24h to do it when you want.

file_id = questions_file.id
batch = openai.batches.create(input_file_id=file_id, endpoint="/v1/chat/completions", completion_window="24h")
batch

Batch(id='batch_6859d8abd654819091a1718b8228f06c', completion_window='24h', created_at=1750718635, endpoint='/v1/chat/completions', input_file_id='file-5zSDHwyFTsKjjFQTfhxG2t', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1750805035, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

## Fourth step - monitor the batch

In [8]:
latest = openai.batches.retrieve(batch.id)
print(latest)
print(latest.status)

Batch(id='batch_6859d8abd654819091a1718b8228f06c', completion_window='24h', created_at=1750718635, endpoint='/v1/chat/completions', input_file_id='file-5zSDHwyFTsKjjFQTfhxG2t', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1750718652, error_file_id=None, errors=None, expired_at=None, expires_at=1750805035, failed_at=None, finalizing_at=1750718649, in_progress_at=1750718636, metadata=None, output_file_id='file-MbrMjVVKxPfk5jtDKpUQEz', request_counts=BatchRequestCounts(completed=9, failed=0, total=9))
completed


## Fifth step - collect the results

In [9]:
results_file_id = latest.output_file_id
results_file = openai.files.content(results_file_id)
results_file.text

'{"id": "batch_req_6859d8b9da588190b4189ec4511c5177", "custom_id": "request-0", "response": {"status_code": 200, "request_id": "25ef90c94bcd88fd073bd891fc3a8b27", "body": {"id": "chatcmpl-BlkBKl7xxvK3pJSa9yNrlMxzYxiHX", "object": "chat.completion", "created": 1750718638, "model": "gpt-4.1-nano-2025-04-14", "choices": [{"index": 0, "message": {"role": "assistant", "content": "The capital of France is Paris.", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 14, "completion_tokens": 7, "total_tokens": 21, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_38343a2f8f"}}, "error": null}\n{"id": "batch_req_6859d8b9e98081908a3f823a1f58d98e", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "4125289f03f01b

## And now let's see

In [10]:
import json
lines = results_file.text.split("\n")
results = [json.loads(line) for line in lines if line]
print(f"Number of results: {len(results)}")
print("First result:")
results[0]

Number of results: 9
First result:


{'id': 'batch_req_6859d8b9da588190b4189ec4511c5177',
 'custom_id': 'request-0',
 'response': {'status_code': 200,
  'request_id': '25ef90c94bcd88fd073bd891fc3a8b27',
  'body': {'id': 'chatcmpl-BlkBKl7xxvK3pJSa9yNrlMxzYxiHX',
   'object': 'chat.completion',
   'created': 1750718638,
   'model': 'gpt-4.1-nano-2025-04-14',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': 'The capital of France is Paris.',
      'refusal': None,
      'annotations': []},
     'logprobs': None,
     'finish_reason': 'stop'}],
   'usage': {'prompt_tokens': 14,
    'completion_tokens': 7,
    'total_tokens': 21,
    'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
    'completion_tokens_details': {'reasoning_tokens': 0,
     'audio_tokens': 0,
     'accepted_prediction_tokens': 0,
     'rejected_prediction_tokens': 0}},
   'service_tier': 'default',
   'system_fingerprint': 'fp_38343a2f8f'}},
 'error': None}

In [11]:
for result in results:
    id = result["custom_id"]
    question = questions[int(id.split("-")[1])]
    print(question)
    print(result["response"]["body"]["choices"][0]["message"]["content"])
    print("-"*100)

What is the capital of France?
The capital of France is Paris.
----------------------------------------------------------------------------------------------------
What is the capital of Germany?
The capital of Germany is Berlin.
----------------------------------------------------------------------------------------------------
What is the capital of Italy?
The capital of Italy is Rome.
----------------------------------------------------------------------------------------------------
What is the capital of Spain?
The capital of Spain is Madrid.
----------------------------------------------------------------------------------------------------
What is the capital of Portugal?
The capital of Portugal is Lisbon.
----------------------------------------------------------------------------------------------------
What is the capital of Greece?
The capital of Greece is Athens.
----------------------------------------------------------------------------------------------------
What is the

## Anthropic has a similar Batch API

https://docs.anthropic.com/en/docs/build-with-claude/batch-processing